<a href="https://colab.research.google.com/github/sarahwking/blank-app/blob/main/sking_portfolioweek9_hinf4220.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import sqlite3
import pandas as pd
import plotly.express as px
import random
from datetime import datetime, timedelta

# Step 1: Create SQLite Database (In-Memory for Performance)
conn = sqlite3.connect(':memory:')
cursor = conn.cursor()

# Step 2: Define Schema
# Fact Table: Stores patient experience data
cursor.execute('''
    CREATE TABLE fact_patient_experience (
        encounter_id INTEGER PRIMARY KEY,
        patient_id INTEGER,
        check_in_time TEXT,
        procedure_start_time TEXT,
        actual_duration INTEGER,
        expected_duration INTEGER,
        satisfaction_score REAL
    );
''')

# Dimension Table: Stores categorical time intervals for analysis
cursor.execute('''
    CREATE TABLE dim_time_intervals (
        time_category_id INTEGER PRIMARY KEY,
        expected_duration_range TEXT,
        actual_duration_range TEXT
    );
''')

# Step 3: Generate Mock Data (Simulating a Typical Week at a Cardiology Clinic)
data = []
start_date = datetime(2025, 3, 1, 8, 0, 0)  # Clinic opens at 8 AM
patient_id = 100
encounter_id = 1

for day in range(5):  # 5 workdays
    for _ in range(40):  # 40 patients per day
        check_in = start_date + timedelta(minutes=random.randint(0, 240))  # Patients arrive over 4 hours
        procedure_start = check_in + timedelta(minutes=random.randint(10, 60))  # Start time varies
        actual_duration = random.randint(20, 50)  # Procedures last between 20-50 minutes
        expected_duration = 30  # Standard expected duration
        satisfaction_score = round(random.uniform(2.5, 5.0), 1)  # Scores between 2.5 - 5.0
        data.append((encounter_id, patient_id, check_in.strftime('%Y-%m-%d %H:%M:%S'), procedure_start.strftime('%Y-%m-%d %H:%M:%S'), actual_duration, expected_duration, satisfaction_score))
        patient_id += 1
        encounter_id += 1
    start_date += timedelta(days=1)  # Move to the next day

# Step 4: Insert Data into the Database
cursor.executemany('''
    INSERT INTO fact_patient_experience (encounter_id, patient_id, check_in_time, procedure_start_time, actual_duration, expected_duration, satisfaction_score)
    VALUES (?, ?, ?, ?, ?, ?, ?);
''', data)
conn.commit()

# Step 5: Run SQL Queries to Extract Metrics
# Query 1: Calculate Average Wait Time Per Day
query_avg_wait_time = '''
    SELECT strftime('%Y-%m-%d', check_in_time) AS visit_date,
           AVG((JULIANDAY(procedure_start_time) - JULIANDAY(check_in_time)) * 1440) AS avg_wait_time
    FROM fact_patient_experience
    GROUP BY visit_date
    ORDER BY visit_date;
'''
avg_wait_time = pd.read_sql(query_avg_wait_time, conn)

# Query 2: Calculate Procedure Efficiency Ratio
query_efficiency_ratio = '''
    SELECT (COUNT(CASE WHEN actual_duration <= expected_duration THEN 1 END) * 100.0 / COUNT(*)) AS efficiency_ratio
    FROM fact_patient_experience;
'''
efficiency_ratio = pd.read_sql(query_efficiency_ratio, conn)

# Query 3: Retrieve Patient Satisfaction Scores
query_satisfaction = '''
    SELECT satisfaction_score FROM fact_patient_experience;
'''
satisfaction_scores = pd.read_sql(query_satisfaction, conn)

# Step 6: Create Visualizations
# Visualization 1: Average Wait Time Per Day
fig1 = px.bar(avg_wait_time, x='visit_date', y='avg_wait_time', title='Average Patient Wait Time Per Day (Minutes)')
fig1.show()

# Fetch Efficiency Ratio Value
efficiency_value = efficiency_ratio.iloc[0, 0]  # Extract efficiency percentage
inefficient_value = 100 - efficiency_value  # Compute inefficient percentage

# Create a DataFrame for Pie Chart
efficiency_data = pd.DataFrame({
    'Category': ['Efficient Procedures', 'Inefficient Procedures'],
    'Percentage': [efficiency_value, inefficient_value]
})

# Visualization 2: Procedure Efficiency Ratio
fig2 = px.pie(efficiency_data, values='Percentage', names='Category', title='Procedure Efficiency Ratio')
fig2.show()

# Visualization 3: Patient Satisfaction Score Distribution
fig3 = px.histogram(satisfaction_scores, x='satisfaction_score', title='Patient Satisfaction Score Distribution')
fig3.show()

# Step 7: Close Database Connection
conn.close()
